In [1]:
import pandas as pd

In [2]:
# importing the revenue data and storing it as a pandas dataframe
rev_filename = "revenue/revenue.csv"
df_rev = pd.read_csv(rev_filename)

In [3]:
# rename the columns of the dataframe to make them more readable
df_rev.rename(columns={'District Number': 'LEA_num', 'District Name': 'LEA', 'Location School Id':
                           'school_ID', 'Location School Number': 'school_num', 'Location Name': 'school',
                           'Fund Code': 'fund', 'Revenue Code': 'function', 'Program Code': 'program',
                           'Amount': 'amt', 'Year': 'year'}, inplace=True)

# remove rows from revenue dataframe that include missing data
df_rev.dropna(axis=0, how='all', inplace=True)

# add a column called "revenue" which designates that each fund listed is a revenue fund, not an expense fund
df_rev['revenue'] = True

In [4]:
df_rev.head()

,LEA_num,LEA,school_ID,school_num,school,fund,function,Revenue Source Two Digit,program,amt,year,n,Unnamed: 12,revenue
0,001,Alpine District,122.0,001,Alpine District,10.0,1110.0,1000.0,5.0,30714587.17,2016.0,1.0,Y,True
1,001,Alpine District,122.0,001,Alpine District,10.0,1111.0,1000.0,5.0,1897475.78,2016.0,2.0,Y,True
2,001,Alpine District,122.0,001,Alpine District,10.0,1112.0,1000.0,5.0,23690571.60,2016.0,3.0,Y,True
3,001,Alpine District,122.0,001,Alpine District,10.0,1113.0,1000.0,5.0,1463548.44,2016.0,4.0,Y,True
4,001,Alpine District,122.0,001,Alpine District,10.0,1114.0,1000.0,5.0,12062125.79,2016.0,5.0,Y,True


In [5]:
# remove irrelevant columns
df_rev.drop(["LEA_num", "Revenue Source Two Digit", "Unnamed: 12", "n"], axis=1, inplace=True)

In [6]:
# import expense data, perform the same steps from revenue data
exp_filename = "expense/expense.csv"
df_exp = pd.read_csv(exp_filename)

C:\Users\Grayson\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
df_exp.rename(columns={'District Number': 'LEA_num', 'District Name': 'LEA', 'Location School Id':
    'school_ID', 'Location School Number': 'school_num', 'Location Name': 'school',
                       'Fund Code': 'fund', 'Function Code': 'function', 'Program Code': 'program',
                       'Amount': 'amt', 'Year': 'year'}, inplace=True)

df_exp.dropna(axis=0, how='all', inplace=True)

df_exp['revenue'] = False

In [8]:
df_exp.head()

,LEA_num,LEA,school_ID,school_num,school,fund,function,Object Code,program,amt,year,revenue
0,001,Alpine District,122,001,Alpine District,10,1000,131,5.0,1.155745e+08,2016,False
1,001,Alpine District,122,001,Alpine District,10,1000,131,1205.0,1.301380e+07,2016,False
2,001,Alpine District,122,001,Alpine District,10,1000,131,1210.0,6.916244e+05,2016,False
3,001,Alpine District,122,001,Alpine District,10,1000,131,1278.0,2.964000e+05,2016,False
4,001,Alpine District,122,001,Alpine District,10,1000,131,5331.0,5.098589e+04,2016,False


In [9]:
df_exp.drop(['LEA_num', 'Object Code'], axis = 1, inplace=True)

In [10]:
# combine both the expense and revenue datasets together
df = df_rev.append(df_exp)

# convert each string into ALL CAPS to prevent issues with key/value mapping
df = df.applymap(lambda s: s.upper() if type(s) == str else s)

In [11]:
# import a conversion table between LEA names and their respective ID values
LEA_keys = "district_keys.csv"
df_LEA_keys = pd.read_csv(LEA_keys)

,LEA,ID
0,ACADEMY FOR MATH ENGINEERING & SCIENCE,110575
1,ADVANTAGE ARTS ACADEMY,186506
2,ALPINE DISTRICT,122
3,AMERICAN ACADEMY OF INNOVATION,186429
4,AMERICAN LEADERSHIP ACADEMY,165615
...,...,...
155,AMERICAN INTERNATIONAL SCHOOL OF UTAH,-1
156,CAPSTONE CLASSICAL ACADEMY,-1
157,DIXIE MONTESSORI ACADEMY,-1
158,INTECH COLLEGIATE HIGH SCHOOL,-1


In [12]:
# convert ID values to integers to remove trailing zeroes
df_LEA_keys['ID'] = df_LEA_keys['ID'].astype('int').astype('str')
df_LEA_keys = df_LEA_keys.applymap(lambda s: s.upper() if type(s) == str else s)

In [13]:
# convert each string into ALL CAPS to prevent issues with key/value mapping
df_LEA_keys = pd.Series(df_LEA_keys['ID'].values, index=df_LEA_keys['LEA']).to_dict()

In [14]:
# using the new conversion table, add LEA_ID to the expense/revenue dataframe
df['LEA_ID'] = df['LEA']
df['LEA_ID'] = df['LEA_ID'].map(df_LEA_keys)

In [15]:
# do the same conversions with school names to school IDs
school_keys = "school_keys.csv"
df_school_keys = pd.read_csv(school_keys).astype(str)
df_school_keys = df_school_keys.applymap(lambda s: s.upper())

In [16]:
df_school_keys = pd.Series(df_school_keys['ID'].values, index=df_school_keys['School']).to_dict()

In [17]:
df['school_ID'] = df['school']
df['school_ID'] = df['school'].map(df_school_keys)

In [18]:
# export the financial data to 'financial.csv'
df.to_csv('financial.csv')

In [19]:
# identify the percentage of null school IDs
df['school_ID'].isnull().sum() * 100 / len(df['school_ID'])

0.28985734933702006

In [20]:
# read in RISE proficiency data into a pandas dataframe
df_RISE = pd.read_csv('proficiency/rise/rise_total.csv')

# rename columns for readability
df_RISE = df_RISE[['year', 'test', 'school',
       'LEA', 'LEA_ID', 'subject',
       'proficiency']]

# remove missing data
df_RISE.dropna(axis=0, how='all', inplace=True)

# convert datatypes
conversions = {'year': int, 'test': str, 'school': str, 'LEA': str, 'LEA_ID': str, 'subject': str, 'proficiency': float}
df_RISE = df_RISE.astype(conversions)

# convert each string into ALL CAPS to prevent issues with key/value mapping
df_RISE = df_RISE.applymap(lambda s: s.upper() if type(s) == str else s)

In [21]:
# read in the SAGE data and perform the same conversions
df_SAGE = pd.read_csv('proficiency/sage.csv')
df_SAGE.dropna(axis=0, how='all', inplace=True)
conversions = {'year': int, 'test': str, 'school': str, 'LEA': str, 'subject': str, 'proficiency': float}
df_SAGE = df_SAGE.astype(conversions)
df_SAGE = df_SAGE.applymap(lambda s: s.upper() if type(s) == str else s)

In [22]:
df_RISE.dtypes

year             int64
test            object
school          object
LEA             object
LEA_ID          object
subject         object
proficiency    float64
dtype: object

In [23]:
# map the school and LEA names to their respective IDs from the conversion tables we used earlier
df_SAGE['school_ID'] = df_SAGE['school']
df_SAGE['school_ID'] = df_SAGE['school'].map(df_school_keys)

df_RISE['school_ID'] = df_RISE['school']
df_RISE['school_ID'] = df_RISE['school'].map(df_school_keys)

df_SAGE['LEA_ID'] = df_SAGE['LEA']
df_SAGE['LEA_ID'] = df_SAGE['LEA'].map(df_LEA_keys)

df_RISE['LEA_ID'] = df_RISE['LEA']
df_RISE['LEA_ID'] = df_RISE['LEA'].map(df_LEA_keys)

In [24]:
# identify the percent of missing school/LEA IDs and names
print(df_SAGE['school_ID'].isnull().sum() * 100 / len(df_SAGE['school_ID']))
print(df_RISE['school_ID'].isnull().sum() * 100 / len(df_RISE['school_ID']))

print(df_SAGE['LEA_ID'].isnull().sum() * 100 / len(df_SAGE['LEA_ID']))
print(df_RISE['LEA_ID'].isnull().sum() * 100 / len(df_RISE['LEA_ID']))

1.3778643103190056
0.1979544704717915
0.3504567919724427
0.0


In [25]:
# aggregate the data for each LEA so that we have total LEA proficiency data to match with revenue/expense data at the LEA level
df_SAGE_agg = df_SAGE.groupby(['LEA', 'year', 'LEA_ID', 'subject', 'test'])['proficiency'].mean().unstack('subject').reset_index()
df_SAGE_agg['school'] = df_SAGE_agg['LEA']
df_SAGE_agg['school_ID'] = df_SAGE_agg['school'].map(df_school_keys)
df_RISE_agg = df_RISE.groupby(['LEA', 'year', 'LEA_ID', 'subject', 'test'])['proficiency'].mean().unstack('subject').reset_index()
df_RISE_agg['school'] = df_RISE_agg['LEA']
df_RISE_agg['school_ID'] = df_RISE_agg['school'].map(df_school_keys)

In [26]:
df_RISE_agg

subject,LEA,year,LEA_ID,test,ENGLISH LANGUAGE ARTS,MATHEMATICS,SCIENCE,school,school_ID
0,ACADEMY FOR MATH ENGINEERING & SCIENCE,2019,110575,RISE,0.543900,0.386300,NaN,ACADEMY FOR MATH ENGINEERING & SCIENCE,700
1,ACADEMY FOR MATH ENGINEERING & SCIENCE,2021,110575,RISE,0.678161,0.550000,0.505376,ACADEMY FOR MATH ENGINEERING & SCIENCE,700
2,ADVANTAGE ARTS ACADEMY,2021,186506,RISE,0.290780,0.223022,0.337209,ADVANTAGE ARTS ACADEMY,01M
3,ALPINE DISTRICT,2019,122,RISE,0.343374,0.283145,0.303801,ALPINE DISTRICT,1
4,ALPINE DISTRICT,2021,122,RISE,0.496344,0.456673,0.505752,ALPINE DISTRICT,1
...,...,...,...,...,...,...,...,...,...
286,WAYNE DISTRICT,2021,1042,RISE,0.420071,0.406880,0.392804,WAYNE DISTRICT,34
287,WEBER DISTRICT,2019,1050,RISE,0.285675,0.245945,0.253145,WEBER DISTRICT,35
288,WEBER DISTRICT,2021,1050,RISE,0.359681,0.374155,0.378302,WEBER DISTRICT,35
289,WEILENMANN SCHOOL OF DISCOVERY,2019,186157,RISE,0.355000,0.288000,0.319700,WEILENMANN SCHOOL OF DISCOVERY,100


In [27]:
# convert the data into a long format to merge with expense/revenue data
df_SAGE_unstack = df_SAGE.groupby(['LEA', 'school', 'year', 'school_ID', 'LEA_ID', 'subject', 'test'])['proficiency'].mean().unstack('subject').reset_index()
df_RISE_unstack = df_RISE.groupby(['LEA', 'school', 'year', 'school_ID', 'LEA_ID', 'subject', 'test'])['proficiency'].mean().unstack('subject').reset_index()

In [28]:
# combine all of the proficiency data together
df_prof = df_RISE_unstack.append(df_SAGE_unstack)
df_prof = df_prof.append(df_SAGE_agg)
df_prof = df_prof.append(df_RISE_agg)

In [29]:
df_prof

,LEA,school,year,school_ID,LEA_ID,test,ENGLISH LANGUAGE ARTS,MATHEMATICS,SCIENCE
0,ACADEMY FOR MATH ENGINEERING & SCIENCE,ACADEMY FOR MATH ENGINEERING & SCIENCE,2019,700,110575,RISE,0.543900,0.386300,NaN
1,ACADEMY FOR MATH ENGINEERING & SCIENCE,ACADEMY FOR MATH ENGINEERING & SCIENCE,2021,700,110575,RISE,0.678161,0.550000,0.505376
2,ADVANTAGE ARTS ACADEMY,ADVANTAGE ARTS ACADEMY,2021,01M,186506,RISE,0.290780,0.223022,0.337209
3,ALPINE DISTRICT,ALPINE ONLINE SCHOOL,2019,299,122,RISE,0.304300,0.355600,NaN
4,ALPINE DISTRICT,ALPINE ONLINE SCHOOL,2021,299,122,RISE,0.477124,0.364780,0.583333
...,...,...,...,...,...,...,...,...,...
286,WAYNE DISTRICT,WAYNE DISTRICT,2021,34,1042,RISE,0.420071,0.406880,0.392804
287,WEBER DISTRICT,WEBER DISTRICT,2019,35,1050,RISE,0.285675,0.245945,0.253145
288,WEBER DISTRICT,WEBER DISTRICT,2021,35,1050,RISE,0.359681,0.374155,0.378302
289,WEILENMANN SCHOOL OF DISCOVERY,WEILENMANN SCHOOL OF DISCOVERY,2019,100,186157,RISE,0.355000,0.288000,0.319700


In [30]:
# merge the proficiency and financial data together my matching an observation in the financial dataset [LEA_ID, school_ID, year]
# with the same [LEA_ID, school_ID, year] in the proficiency dataset
df = df.merge(df_prof, how='left', on=['LEA_ID', 'school_ID', 'year'])

In [31]:
df

,LEA_x,school_ID,school_num,school_x,fund,function,program,amt,year,revenue,LEA_ID,LEA_y,school_y,test,ENGLISH LANGUAGE ARTS,MATHEMATICS,SCIENCE
0,ALPINE DISTRICT,1,001,ALPINE DISTRICT,10.0,1110.0,5.0,30714587.17,2016.0,True,122,ALPINE DISTRICT,ALPINE DISTRICT,SAGE,0.487232,0.532894,0.555292
1,ALPINE DISTRICT,1,001,ALPINE DISTRICT,10.0,1111.0,5.0,1897475.78,2016.0,True,122,ALPINE DISTRICT,ALPINE DISTRICT,SAGE,0.487232,0.532894,0.555292
2,ALPINE DISTRICT,1,001,ALPINE DISTRICT,10.0,1112.0,5.0,23690571.60,2016.0,True,122,ALPINE DISTRICT,ALPINE DISTRICT,SAGE,0.487232,0.532894,0.555292
3,ALPINE DISTRICT,1,001,ALPINE DISTRICT,10.0,1113.0,5.0,1463548.44,2016.0,True,122,ALPINE DISTRICT,ALPINE DISTRICT,SAGE,0.487232,0.532894,0.555292
4,ALPINE DISTRICT,1,001,ALPINE DISTRICT,10.0,1114.0,5.0,12062125.79,2016.0,True,122,ALPINE DISTRICT,ALPINE DISTRICT,SAGE,0.487232,0.532894,0.555292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000297,SUCCESS ACADEMY,710,710,SUCCESS DSU,10.0,2400.0,5.0,7212.54,2021.0,False,165608,SUCCESS ACADEMY,SUCCESS DSU,RISE,0.673077,NaN,0.634615
1000298,SUCCESS ACADEMY,710,710,SUCCESS DSU,10.0,2400.0,5.0,259.92,2021.0,False,165608,SUCCESS ACADEMY,SUCCESS DSU,RISE,0.673077,NaN,0.634615
1000299,SUCCESS ACADEMY,710,710,SUCCESS DSU,10.0,2400.0,5.0,2542.53,2021.0,False,165608,SUCCESS ACADEMY,SUCCESS DSU,RISE,0.673077,NaN,0.634615
1000300,SUCCESS ACADEMY,710,710,SUCCESS DSU,10.0,2500.0,50.0,590.04,2021.0,False,165608,SUCCESS ACADEMY,SUCCESS DSU,RISE,0.673077,NaN,0.634615


In [32]:
# identify the amount of missing data for each column
for c in df.columns:
    print(str(c) + ": " + str(df[c].isnull().sum() * 100 / len(df[c])))

LEA_x: 0.2582220169508808
school_ID: 0.2582220169508808
school_num: 0.2582220169508808
school_x: 0.2582220169508808
fund: 0.2582220169508808
function: 0.2582220169508808
program: 0.2639202960705867
amt: 0.2582220169508808
year: 0.2582220169508808
revenue: 0.0
LEA_ID: 0.44306619400940916
LEA_y: 24.68694454274809
school_y: 24.68694454274809
test: 24.68694454274809
ENGLISH LANGUAGE ARTS: 26.22248081079514
MATHEMATICS: 26.380733018628373
SCIENCE: 28.22527596665807


In [33]:
# export the data to a csv
df.to_csv("long_merge.csv")